# Modelos de clasificación con datos numéricos, imágenes y mixto

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# Copiamos los ficheros de GDrive al entorno de ejecución
!cp /content/drive/My\ Drive/x_train_num.npy x_train_num.npy
!cp /content/drive/My\ Drive/x_train_img.npy x_train_img.npy
!cp /content/drive/My\ Drive/x_val_num.npy x_val_num.npy
!cp /content/drive/My\ Drive/x_val_img.npy x_val_img.npy
!cp /content/drive/My\ Drive/x_test_num.npy x_test_num.npy
!cp /content/drive/My\ Drive/x_test_img.npy x_test_img.npy
!cp /content/drive/My\ Drive/y_train_class.npy y_train_class.npy
!cp /content/drive/My\ Drive/y_val_class.npy y_val_class.npy
!cp /content/drive/My\ Drive/y_test_class.npy y_test_class.npy

In [3]:
# Cargamos los ficheros a variables de nuestro entorno
import numpy as np
x_train_num = np.load('x_train_num.npy')
x_train_img = np.load('x_train_img.npy')
x_val_num = np.load('x_val_num.npy')
x_val_img = np.load('x_val_img.npy')
x_test_num = np.load('x_test_num.npy')
x_test_img = np.load('x_test_img.npy')
y_train_class = np.load('y_train_class.npy')
y_val_class = np.load('y_val_class.npy')
y_test_class = np.load('y_test_class.npy')

In [4]:
# Mostramos la forma de los arrays para ver que no haya errores
print("datos numericos de train:", x_train_num.shape)
print("datos de imágenes de train:",x_train_img.shape)
print("etiquetas de train:",y_train_class.shape)
print("datos numericos de validación:",x_val_num.shape)
print("datos de imágenes de validación:",x_val_img.shape)
print("etiquetas de validación:",y_val_class.shape)
print("datos numericos de test:",x_test_num.shape)
print("datos de imágenes de test:",x_test_img.shape)
print("etiquetas de test:",y_test_class.shape)

datos numericos de train: (7125, 30)
datos de imágenes de train: (7125, 64, 64, 3)
etiquetas de train: (7125, 3)
datos numericos de validación: (814, 30)
datos de imágenes de validación: (814, 64, 64, 3)
etiquetas de validación: (814, 3)
datos numericos de test: (2687, 30)
datos de imágenes de test: (2687, 64, 64, 3)
etiquetas de test: (2687, 3)


### Modelo de clasificación sólo con datos numéricos

Creamos un modelo secuencial con una primera capa densa con 8 neuronas. En esa primera capa hay que pasar la dimensión y la función de activación es relu. La siguiente capa que añadimos ya es más pequeña con 4 neuronas y también función de activación relu. La última capa al ser una red para clasificación tiene tantas neuronas como grupos hay, es decir, 3. Y su función de activación es softmax.

In [7]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(8, input_dim=x_train_num.shape[1], activation="relu"))
model.add(Dense(4, activation="relu"))
model.add(Dense(3, activation="softmax"))

Vamos a usar el optimizador Adam y la función de pérdidas la crossentropía.

In [8]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])
# train the model
print("[INFO] training model...")
model.fit(x=x_train_num, y=y_train_class, 
	validation_data=(x_val_num, y_val_class),
	epochs=50, batch_size=8)

[INFO] training model...
Epoch 1/50
891/891 [==============================] - 3s 4ms/step - loss: 5.9831 - accuracy: 0.3864 - val_loss: 1.0927 - val_accuracy: 0.4410
Epoch 2/50
891/891 [==============================] - 3s 3ms/step - loss: 1.0789 - accuracy: 0.4087 - val_loss: 1.0851 - val_accuracy: 0.4373
Epoch 3/50
891/891 [==============================] - 3s 4ms/step - loss: 1.0651 - accuracy: 0.4124 - val_loss: 1.0656 - val_accuracy: 0.4509
Epoch 4/50
891/891 [==============================] - 3s 4ms/step - loss: 1.0531 - accuracy: 0.4250 - val_loss: 1.0477 - val_accuracy: 0.4570
Epoch 5/50
891/891 [==============================] - 3s 3ms/step - loss: 1.0160 - accuracy: 0.4640 - val_loss: 0.9262 - val_accuracy: 0.5983
Epoch 6/50
891/891 [==============================] - 3s 4ms/step - loss: 0.8354 - accuracy: 0.6514 - val_loss: 0.7372 - val_accuracy: 0.7039
Epoch 7/50
891/891 [==============================] - 3s 3ms/step - loss: 0.7447 - accuracy: 0.6887 - val_loss: 0.7211 - va

Comprobamos el modelo con los datos de test.

In [9]:
scores = model.evaluate(x_test_num, y_test_class, verbose=0)
print('Loss: %.3f' % scores[0])
print('Accuracy: %.3f' % scores[1])

Loss: 0.602
Accuracy: 0.757


### Modelo de clasificación sólo con imágenes

En esta ocación usaremos una red neuronal convolucional para poder trabajar con las imágenes. Crearemos el modelo mediante un método funcional en vez de secuencial como anteriormente. Dependiendo de los filtros que pasemos creará diferentes capas, en este caso crearemos una de 16, otra de 32 y otra de 64. En la primera iteración indicamos la dimensión de los datos al igual que hacíamos en el modelo secuencial.

In [10]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input
# Tamaño de las imágenes
inputShape = (64, 64, 3)
chanDim = -1
# Filtros que vamos a aplicar en las capas convolucionales
filters=(16, 32, 64)
inputs = Input(shape=inputShape)
# Recorremos el bucle creando las diferentes capas
for (i, f) in enumerate(filters):
		# si es la primera capa indicamos el inputshape
  if i == 0:
	  x = inputs
	# CONV => BN => RELU => POOL
  x = Conv2D(f, (3, 3), padding="same")(x)
  x = BatchNormalization(axis=chanDim)(x)
  x = Activation("relu")(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)
	
# Aplanamos los datos y ponemos una capa densa de 16 neuronas con activación relu
x = Flatten()(x)
x = Dense(16)(x)
x = BatchNormalization(axis=chanDim)(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
# Añadimos otra capa densa de 4 neuronas y activación relu
x = Dense(4)(x)
x = Activation("relu")(x)
# Añadimos la última capa con 3 neuronas y función softmax para la calsificación 
x = Dense(3, activation="softmax")(x)
# Construimos la red neuronal
model = Model(inputs, x)

Vamos a usar el optimizador Adam y la función de pérdidas la crossentropía.

In [11]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])
# entrenamos el modelo
print("[INFO] training model...")
model.fit(x=x_train_img, y=y_train_class, 
    validation_data=(x_val_img, y_val_class),
    epochs=50, batch_size=8)

[INFO] training model...
Epoch 1/50
891/891 [==============================] - 8s 9ms/step - loss: 1.0719 - accuracy: 0.4209 - val_loss: 1.1054 - val_accuracy: 0.3354
Epoch 2/50
891/891 [==============================] - 7s 8ms/step - loss: 1.0422 - accuracy: 0.4521 - val_loss: 1.0268 - val_accuracy: 0.4779
Epoch 3/50
891/891 [==============================] - 7s 8ms/step - loss: 1.0259 - accuracy: 0.4643 - val_loss: 1.0192 - val_accuracy: 0.4681
Epoch 4/50
891/891 [==============================] - 7s 8ms/step - loss: 1.0115 - accuracy: 0.4741 - val_loss: 1.0470 - val_accuracy: 0.4644
Epoch 5/50
891/891 [==============================] - 7s 8ms/step - loss: 1.0115 - accuracy: 0.4813 - val_loss: 1.0653 - val_accuracy: 0.4459
Epoch 6/50
891/891 [==============================] - 7s 8ms/step - loss: 1.0016 - accuracy: 0.4859 - val_loss: 1.1027 - val_accuracy: 0.4607
Epoch 7/50
891/891 [==============================] - 7s 8ms/step - loss: 0.9913 - accuracy: 0.4900 - val_loss: 1.0111 - va

Comprobamos el modelo con los datos de test.

In [12]:
scores = model.evaluate(x_test_img, y_test_class, verbose=0)
print('Loss: %.3f' % scores[0])
print('Accuracy: %.3f' % scores[1])

Loss: 1.974
Accuracy: 0.469


### Modelo de clasificación sólo con datos mixtos, imágenes y numéricos.

En esta ocasión usaremos los dos tipos de datos que hemos visto anteriormente. Vamos a crear un modelo con dos ramas:


1.   Rama 1: una red neuronal para analizar los datos numéricos, exactamente igual a la que hemos usado antes, pero sin la capa final de clasificación.
2.   Rama 2: una red neuronal convolucional para trabajar con las imágenes, exactamente igual a la anterior, pero sin la capa final de clasificación.

Posteriormente combinaremos ambas ramas creando otra capa densa de 4 neuronas con función de activación relu y la capa final de clasificación con 3 neuronas y función de activación softmax.


In [13]:
# Creamos la rama numérica
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

mlp = Sequential()
mlp.add(Dense(8, input_dim=x_train_num.shape[1], activation="relu"))
mlp.add(Dense(4, activation="relu"))

In [14]:
# Creamos la rama que trabaja con imágenes
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input
# Tamaño de las imágenes
inputShape = (64, 64, 3)
chanDim = -1
# Filtros que vamos a aplicar en las capas convolucionales
filters=(16, 32, 64)
inputs = Input(shape=inputShape)
# Recorremos el bucle creando las diferentes capas
for (i, f) in enumerate(filters):
		# si es la primera capa indicamos el inputshape
  if i == 0:
	  x = inputs
	# CONV => BN => RELU => POOL
  x = Conv2D(f, (3, 3), padding="same")(x)
  x = BatchNormalization(axis=chanDim)(x)
  x = Activation("relu")(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)
	
# Aplanamos los datos y ponemos una capa densa de 16 neuronas con activación relu
x = Flatten()(x)
x = Dense(16)(x)
x = BatchNormalization(axis=chanDim)(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
# Añadimos otra capa densa de 4 neuronas y activación relu
x = Dense(4)(x)
x = Activation("relu")(x)
# Construimos la red neuronal
cnn = Model(inputs, x)

In [15]:
from tensorflow.keras.layers import concatenate
# Combinamos ambas redes
combinedInput = concatenate([mlp.output, cnn.output])

In [16]:
# Añadimos la capa densa con 4 neuronas y la de 3 para la clasificación
x = Dense(4, activation="relu")(combinedInput)
x = Dense(3, activation="softmax")(x)

In [17]:
# Construimos el modelo que combina datos numéricos e imágenes
model = Model(inputs=[mlp.input, cnn.input], outputs=x)

In [18]:
# Usamos de nuevo el optimizados Adam y la función de pérdidas crossentropía y compilamos el modelo
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])

In [19]:
# Entrenamos el modelo
print("[INFO] training model...")
model.fit(
	x=[x_train_num, x_train_img], y=y_train_class,
	validation_data=([x_val_num, x_val_img], y_val_class),
	epochs=50, batch_size=8)


[INFO] training model...
Epoch 1/50
891/891 [==============================] - 8s 9ms/step - loss: 1.6955 - accuracy: 0.3641 - val_loss: 1.0795 - val_accuracy: 0.4459
Epoch 2/50
891/891 [==============================] - 8s 9ms/step - loss: 1.0763 - accuracy: 0.4146 - val_loss: 1.0763 - val_accuracy: 0.4459
Epoch 3/50
891/891 [==============================] - 8s 9ms/step - loss: 1.0687 - accuracy: 0.4225 - val_loss: 1.0312 - val_accuracy: 0.5135
Epoch 4/50
891/891 [==============================] - 8s 9ms/step - loss: 1.0890 - accuracy: 0.4469 - val_loss: 1.0650 - val_accuracy: 0.4619
Epoch 5/50
891/891 [==============================] - 8s 9ms/step - loss: 1.0607 - accuracy: 0.4389 - val_loss: 1.0618 - val_accuracy: 0.4631
Epoch 6/50
891/891 [==============================] - 8s 9ms/step - loss: 1.0534 - accuracy: 0.4484 - val_loss: 1.0567 - val_accuracy: 0.4754
Epoch 7/50
891/891 [==============================] - 8s 9ms/step - loss: 1.0443 - accuracy: 0.4578 - val_loss: 1.0403 - va

Comprobamos el modelo con los datos de test.

In [20]:
scores = model.evaluate([x_test_num, x_test_img], y_test_class, verbose=0)
print('Loss: %.3f' % scores[0])
print('Accuracy: %.3f' % scores[1])

Loss: 0.747
Accuracy: 0.732


Podemos concluir que las imágenes aportan muy poco (o nada) al modelo y que simplemente con los datos numéricos tenemos mejor precisión.